In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import configparser
sys.path.insert(0, 'Utils/')

from AISDataManager import AISDataManager
import SimpleUtils as sU
import Constants as c
import TimeUtils as timeUtils
import HMUtils as hMUtil
import GeoCompute as gC

aISDM = AISDataManager()

config = configparser.ConfigParser()
config.read('DefaultConfig.INI')

In [ ]:
lonMin = (float)(config['LSTM_MULTI_TASK_DEST_TRAIN']['LON_MIN'])
lonMax = (float)(config['LSTM_MULTI_TASK_DEST_TRAIN']['LON_MAX'])

latMin = (float)(config['LSTM_MULTI_TASK_DEST_TRAIN']['LAT_MIN'])
latMax = (float)(config['LSTM_MULTI_TASK_DEST_TRAIN']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

In [ ]:
class VesselTypeSource:
    """
    The VesselTypeSource object contains information. 
    which is useful for the training data
    """
    def __init__(self, srcDir, typeVes):
        self.srcDir = srcDir
        self.type = typeVes
        self.trainList = []

In [ ]:
#add more sources here based on requirement
sourceDir1 = config['LSTM_MULTI_TASK_DEST_TRAIN']['SRC_DIR_1']
sourceDir1Test = config['LSTM_MULTI_TASK_DEST_TRAIN']['SRC_DIR_TEST_1']
sourceDir2 = config['LSTM_MULTI_TASK_DEST_TRAIN']['SRC_DIR_2']
sourceDir2Test = config['LSTM_MULTI_TASK_DEST_TRAIN']['SRC_DIR_TEST_2']

modelDir = config['LSTM_MULTI_TASK_DEST_TRAIN']['MODEL_DIR']
modelTS = int(config['LSTM_MULTI_TASK_DEST_TRAIN']['MODEL_TS'])
prevTS = int(60/modelTS)+1
#in minutes
dataMinimumUnit = 5
print(sourceDir1)
print(sourceDir2)
print(modelDir)
print(prevTS)
tsSkip = int(modelTS/dataMinimumUnit) - 1
print(tsSkip)

In [ ]:
vesselSource1 = VesselTypeSource(sourceDir1,0)
vesselSource2 = VesselTypeSource(sourceDir2,1)

vesselSource1Test = VesselTypeSource(sourceDir1Test,0)
vesselSource2Test = VesselTypeSource(sourceDir2Test,1)

vesselDataSources = [vesselSource1, vesselSource2]
vesselDataSourcesTest = [vesselSource1Test, vesselSource2Test]
print(vesselDataSources[0])
print(vesselDataSources[1])

In [ ]:
def get_traj_data(fileName):
    """
    """
    #TODO:uncomment/comment next two lines
#     sourceDF,_ = aISDM.load_data_from_csv(fileName)
    sourceDF,_ = aISDM.load_data_from_csv(fileName.replace("jcharla","jagir"))
    return sourceDF.to_numpy()

In [ ]:
def convert_seq_to_x_y(seq, vType, prevTimeStamp):
    """
    """
    #-prevTimeStamp is is to take care of boundary condition
    #since we are considering prevTimeStamp time stamps for the input data
    xNumRows = seq[:-(prevTimeStamp),:].shape[0]
    print(xNumRows)
    dateStr = seq[0][0]
    monF = int(dateStr.split("-")[1])
    print(monF)
    monData = np.zeros((xNumRows, 12))
    monData[:,(12-monF)] = 1
    
    #TODO: change 2 for multiple types
    typeArr = np.zeros((xNumRows,2))
    typeArr[:,vType] = 1
    
    lonLatColList = []
    for start in range(prevTimeStamp):
        lonLatColList.append(seq[start:(-prevTimeStamp+start),1:].copy())
        
    outputLabelLonLat = seq[prevTimeStamp:,[1,2,3,4]].copy()
    
    outputLabel = (outputLabelLonLat)
    
    xDataTS = np.zeros((xNumRows,0))
    
    for tS in range(prevTimeStamp):
        xDataTS = np.hstack((xDataTS,lonLatColList[tS]))
        xDataTS = np.hstack((xDataTS,typeArr))
        xDataTS = np.hstack((xDataTS,monData))
    
    return xDataTS, outputLabel

In [ ]:
#prepare data in x, y formate
#this is memory intensive task may take a time
#LAT, LON, SOG, COG, LAT_DEST, LON_DEST, Month (12), Type (2)
tSCol = 20
tSCol = tSCol * prevTS
xDataTS = np.zeros((0,tSCol))
yData = np.zeros((0,4))
# for vesselDataSource in vesselDataSources[0:1]:
for vesselDataSource in vesselDataSources[0:]:
    print("Taking data from:%s"%vesselDataSource.srcDir)
    mMSIListFile = vesselDataSource.srcDir + 'TrainTrajCount.txt'
    mMSIList = [line.rstrip('\n') for line in open(mMSIListFile)]
#     for traj in mMSIList[0:1]:
    for traj in mMSIList[0:]:
        numTraj = traj.split("_")[-1]
        trajAffix = traj[0:-len(numTraj)]
        for num in range(int(numTraj)):
            trajName = trajAffix + str(num) +'.csv'
            rawData = get_traj_data(trajName)
#             print(rawData.shape)
            for trajSkip in range(tsSkip+1):
                selectIDX = np.arange(trajSkip, rawData.shape[0], (tsSkip+1))
#                 print(selectIDX)
                selectedData = rawData[selectIDX,:]
#                 print(selectedData)
                xTSTemp, yTemp = convert_seq_to_x_y(selectedData, vesselDataSource.type, prevTS)
#                 print(xTSTemp)
                xDataTS = np.vstack((xDataTS,xTSTemp.copy()))
                yData = np.vstack((yData,yTemp.copy()))

In [ ]:
print(xDataTS)
print(yData)

In [ ]:
print(xDataTS.shape)
print(yData.shape)

In [ ]:
#save prepared data
xTSToStore = modelDir + "XDataTS.npy"
yToStore = modelDir + "YData.npy"
np.save(xTSToStore, xDataTS)
np.save(yToStore, yData)


In [ ]:
#data can be loaded directly, if it has been prepared earlier and saved
numTSFeature = 20
xTSToStore = modelDir + "XDataTS.npy"
yToStore = modelDir + "YData.npy"

xDataTS = np.load(xTSToStore, allow_pickle=True)
yData = np.load(yToStore, allow_pickle=True)

In [ ]:
print(xDataTS.shape)
print(yData.shape)

In [ ]:
#Computing Min and max for SOG
sogFirstCol = 2
nextSOGCol = sogFirstCol
sOGMin = np.min(xDataTS[:,nextSOGCol])
sOGMax = np.max(xDataTS[:,nextSOGCol])
while(nextSOGCol < xDataTS.shape[1]):
    tempMinSOG = np.min(xDataTS[:,nextSOGCol])
    tempMaxSOG = np.max(xDataTS[:,nextSOGCol])
    if(tempMinSOG < sOGMin):
        sOGMin = tempMinSOG
        
    if(tempMaxSOG > sOGMax):
        sOGMax = tempMaxSOG
        
    nextSOGCol = nextSOGCol + numTSFeature
print(sOGMin, sOGMax)

In [ ]:
#Computing Min and max for COG
cogFirstCol = 3
nextCOGCol = cogFirstCol
cOGMin = np.min(xDataTS[:,nextCOGCol])
cOGMax = np.max(xDataTS[:,nextCOGCol])
while(nextCOGCol < xDataTS.shape[1]):
    tempMinCOG = np.min(xDataTS[:,nextCOGCol])
    tempMaxCOG = np.max(xDataTS[:,nextCOGCol])
    if(tempMinCOG < cOGMin):
        cOGMin = tempMinCOG
        
    if(tempMaxCOG > cOGMax):
        cOGMax = tempMaxCOG
        
    nextCOGCol = nextCOGCol + numTSFeature
print(cOGMin, cOGMax)

In [ ]:
print(xDataTS[0,:])
print(yData[0,:])
print(xDataTS[1,:])
print(yData[1,:])

In [ ]:
#Normalise the data
xDataTSNorm = xDataTS.copy()
colAccess = 0
for prevTime in range(prevTS):
    xDataTSNorm[:,(colAccess) + 0] = (xDataTS[:,(colAccess) + 0] - latMin)/(latMax - latMin)
    xDataTSNorm[:,(colAccess) + 1] = (xDataTS[:,(colAccess) + 1] - lonMin)/(lonMax - lonMin)
    xDataTSNorm[:,(colAccess) + 2] = (xDataTS[:,(colAccess) + 2] - sOGMin)/(sOGMax - sOGMin)
    xDataTSNorm[:,(colAccess) + 3] = (xDataTS[:,(colAccess) + 3] - cOGMin)/(cOGMax - cOGMin)
    xDataTSNorm[:,(colAccess) + 4] = (xDataTS[:,(colAccess) + 4] - latMin)/(latMax - latMin)
    xDataTSNorm[:,(colAccess) + 5] = (xDataTS[:,(colAccess) + 5] - lonMin)/(lonMax - lonMin)
    colAccess = colAccess + numTSFeature

    
xDataTSNorm = np.reshape(xDataTSNorm,(xDataTSNorm.shape[0], prevTS, numTSFeature))

In [ ]:
#normalise the output data as well
yLonLatData_0 = (yData[:,0] - latMin)/(latMax - latMin)
yLonLatData_0 = np.reshape(yLonLatData_0,(yLonLatData_0.shape[0],1))
yLonLatData_1 = (yData[:,1] - lonMin)/(lonMax - lonMin)
yLonLatData_1 = np.reshape(yLonLatData_1,(yLonLatData_1.shape[0],1))
yLonLatData_2 = (yData[:,2] - sOGMin)/(sOGMax - sOGMin)
yLonLatData_2 = np.reshape(yLonLatData_2,(yLonLatData_2.shape[0],1))
yLonLatData_3 = (yData[:,3] - cOGMin)/(cOGMax - cOGMin)
yLonLatData_3 = np.reshape(yLonLatData_3,(yLonLatData_3.shape[0],1))
yDataNorm = np.hstack((yLonLatData_0,yLonLatData_1,yLonLatData_2,yLonLatData_3))

In [ ]:
from keras import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
#define the model
lonLatTS = Input(shape=(prevTS,numTSFeature))
hidden1 = LSTM(50, return_sequences= True)(lonLatTS)
# hidden21 = LSTM(50)(hidden1)
# hidden22 = LSTM(50)(hidden1)
# hidden23 = LSTM(50)(hidden1)
# hidden24 = LSTM(50)(hidden1)
hidden2 = LSTM(50)(hidden1)

latDense1 = Dense(150, activation='relu')(hidden2)
lonDense1 = Dense(150, activation='relu')(hidden2)
sOGDense1 = Dense(150, activation='relu')(hidden2)
cOGDense1 = Dense(150, activation='relu')(hidden2)

latDense2 = Dense(150, activation='relu')(latDense1)
lonDense2 = Dense(150, activation='relu')(lonDense1)
sOGDense2 = Dense(150, activation='relu')(sOGDense1)
cOGDense2 = Dense(150, activation='relu')(cOGDense1)

latOp = Dense(1, activation='linear')(latDense2)
lonOp = Dense(1, activation='linear')(lonDense2)
sOGOp = Dense(1, activation='linear')(sOGDense2)
cOGOp = Dense(1, activation='linear')(cOGDense2)

modelMultiTask = Model(inputs=lonLatTS, outputs=[latOp,lonOp,sOGOp,cOGOp])

In [ ]:
modelMultiTask.summary()

In [ ]:
modelMultiTask.compile(optimizer='adam', loss='mse')

In [ ]:
#train the model
modelMultiTaskHist = modelMultiTask.fit(xDataTSNorm, [yDataNorm[:,0],yDataNorm[:,1],yDataNorm[:,2],yDataNorm[:,3]], epochs=1000, batch_size = 1024 , verbose = 2)

In [ ]:
plt.plot(modelMultiTaskHist.history['loss'])

In [ ]:
modelMultiTaskDir = modelDir + "MODEL_MULTI_TASK_1000_MSE.h5"

In [ ]:
#save the model
modelMultiTask.save(modelMultiTaskDir)

In [ ]:
#load the already saved model
from keras.models import load_model
modelMultiTask = load_model(modelMultiTaskDir)

In [ ]:
def get_traj_data_for_pred(fileName, vType):
    tempSourceDF,_ = aISDM.load_data_from_csv(fileName)
    tempSourceDF = tempSourceDF.to_numpy()
    tempMonth = int(tempSourceDF[0][0].split("-")[1])
    tempType = vType
    selectIDX = np.arange(0, tempSourceDF.shape[0], (tsSkip+1))
    tempSourceDF = tempSourceDF[selectIDX,:]
    tempTraj = tempSourceDF[0:prevTS,1:]
    return tempTraj, tempType, tempMonth

tempTestFile = "/home/jagir/LiporLab/Data/M122_00_M118_50_33_40_36_40/MMSI_1004_TRAIN_WDEST_INTP/355745000_0_0_0.csv"
tempTraj, tempType, tempMonth = get_traj_data_for_pred(tempTestFile, 0)

In [ ]:
print(tempTraj)

In [ ]:
def normalize_traj_data(arr):
    """
    """
    #subtract the minimum 
    #and divide by range
    ret0 = (arr[:,0] - latMin)/(latMax - latMin)
    ret0 = np.reshape(ret0, (ret0.shape[0],1))
    ret1 = (arr[:,1] - lonMin)/(lonMax - lonMin)
    ret1 = np.reshape(ret1, (ret1.shape[0],1))
    ret2 = (arr[:,2] - sOGMin)/(sOGMax - sOGMin)
    ret2 = np.reshape(ret2, (ret2.shape[0],1))
    ret3 = (arr[:,3] - cOGMin)/(cOGMax - cOGMin)
    ret3 = np.reshape(ret3, (ret3.shape[0],1))
    ret4 = (arr[:,4] - latMin)/(latMax - latMin)
    ret4 = np.reshape(ret4, (ret4.shape[0],1))
    ret5 = (arr[:,5] - lonMin)/(lonMax - lonMin)
    ret5 = np.reshape(ret5, (ret5.shape[0],1))
    ret = np.hstack((ret0, ret1, ret2, ret3, ret4, ret5))
    return ret

In [ ]:
def compute_prediction(prevTraj, vType, monthOfTraj):
    """
    computes prediction of a function
    
    Parameter:
        prevTraj:(prevTS)x6 (np array of LAT, LON, SOG, COG, DEST_LAT, DEST_LON)
        vType:type of vessel (0 or 1)
        monthOfTraj: month of trajectory (1 to 12)
    """
    prevTrajNorm = normalize_traj_data(prevTraj)
    
    monData = np.zeros((prevTS, 12))
    monData[:,(12-monthOfTraj)] = 1
    
    typeData = np.zeros((prevTS,2))
    typeData[:,vType] = 1
    
    xTSNorm = np.zeros((prevTS,0))
    xTSNorm = np.hstack((xTSNorm,prevTrajNorm))
    xTSNorm = np.hstack((xTSNorm,typeData))
    xTSNorm = np.hstack((xTSNorm,monData))
    
    xTSNorm = np.reshape(xTSNorm,(1,xTSNorm.shape[0],xTSNorm.shape[1]))
    predLat, predLon, predSOG, predCOG = modelMultiTask.predict(xTSNorm)
    
    #after prediction de normalise it
    predLatScaled = (predLat * (latMax - latMin)) + latMin
    predLonScaled = (predLon * (lonMax - lonMin)) + lonMin
    predSOGScaled = (predSOG * (sOGMax - sOGMin)) + sOGMin
    predCOGScaled = (predCOG * (cOGMax - cOGMin)) + cOGMin
    return predLatScaled, predLonScaled, predSOGScaled, predCOGScaled
    
compute_prediction(tempTraj, tempType, tempMonth)

In [ ]:
def compute_n_unit_pred(prevTraj, vType, monVal, n = 1):
    """
    Compute prediction for n units
    
    calls compute_prediction
    according to value of n
    default value of n = 1
    """
    firstLoc = prevTraj.copy()
    destLAT = prevTraj[0,4]
    destLON = prevTraj[0,5]
    
    #return value will be numpy array of nx4
    ret = np.zeros((0,4))
    for i in range(n):
        #make prediction
        predLat, predLon, predSOG, predCOG = compute_prediction(firstLoc, vType, monVal)
        #append them in ret
        ret = np.vstack((ret,np.array([[predLat.flatten()[0] \
                                        ,predLon.flatten()[0] \
                                        ,predSOG.flatten()[0] \
                                        ,predCOG.flatten()[0]]])))
        
        #update firstLoc 
        #for next  iteration
        firstLoc = firstLoc[1:,:].copy()
        firstLoc = np.vstack((firstLoc,np.array([[predLat.flatten()[0] \
                          , predLon.flatten()[0] \
                          , predSOG.flatten()[0] \
                          , predCOG.flatten()[0] \
                          , destLAT \
                          , destLON]])))
    return ret

compute_n_unit_pred(tempTraj, tempType, tempMonth, 5)

In [ ]:
def get_error_for_traj(trajFile, vesselType):
    """
    Compute error for the entire trajectory.
    
    Returns list of error for predictions of upto 
    4 hours i.e. 240 minutes.
    
    Parameters:
        trajFile (str): file containing trajectory
    Returns:
        errorVal (list of floats): list of errors in KM for each
                                   30 min predictions.
    """    
    errorVal = []
    
    vesselTraj, vType, monData = get_traj_data_for_pred(trajFile, vesselType)        
    numOfPredFor240 = int(240/modelTS)    
    predVesselTraj = compute_n_unit_pred(vesselTraj, vType, monData ,n = numOfPredFor240)    
    vesselActTraj = get_traj_data(trajFile)
    minShapeOfActTraj = int(300/dataMinimumUnit)+1
    
    if(vesselActTraj.shape[0] < (minShapeOfActTraj)):
        #it should not go into this condition
        #Trajetories excluding these can be used as a benchmark dataset
        print("Trajectory not used for error")
        return []
    
    selectIDX = np.arange(0, vesselActTraj.shape[0], (tsSkip+1))
    vesselActTraj = vesselActTraj[selectIDX,:]
    
    for pred in range(predVesselTraj.shape[0]):
        trueLat, trueLon = vesselActTraj[(prevTS + pred),1], vesselActTraj[(prevTS + pred),2]
        predLat, predLon= predVesselTraj[pred,0], predVesselTraj[pred,1]
        errorVal.append(gC.compute_distance(trueLon, trueLat, predLon, predLat))
    
    usefulPredStart = int(30/modelTS)-1
    usefulPredDiff = int(30/modelTS)
    errorRet = []
    for i in range(usefulPredStart, len(errorVal), usefulPredDiff):
        errorRet.append(errorVal[i])
    return errorRet

get_error_for_traj(tempTestFile, 0)

In [ ]:
#compute errors for all training trajectories
trainDataWholeErrors = []
# for vesselDataSource in vesselDataSources[0:1]:
for vesselDataSource in vesselDataSources[0:]:
    print("Taking data from:%s"%vesselDataSource.srcDir)
    mMSIListFile = vesselDataSource.srcDir + 'TrainTrajCount.txt'
    mMSIList = [line.rstrip('\n') for line in open(mMSIListFile)]
#     for traj in mMSIList[0:1]:
    for traj in mMSIList[0:]:
        numTraj = traj.split("_")[-1]
        trajAffix = traj[0:-len(numTraj)]
        for num in range(int(numTraj)):
            trajName = trajAffix + str(num) +'.csv'
            #TODO:.replace statement can be removed in the future
            trainDataWholeErrors.append(get_error_for_traj(trajName.replace("jcharla","jagir"),vesselDataSource.type))

In [ ]:
trainWholeErr_30 = []
trainWholeErr_60 = []
trainWholeErr_90 = []
trainWholeErr_120 = []
trainWholeErr_150 = []
trainWholeErr_180 = []
trainWholeErr_210 = []
trainWholeErr_240 = []

trainWholeErr_n_30 = [trainWholeErr_30 \
, trainWholeErr_60 \
, trainWholeErr_90 \
, trainWholeErr_120 \
, trainWholeErr_150 \
, trainWholeErr_180 \
, trainWholeErr_210 \
, trainWholeErr_240 \
]

for trajErr in trainDataWholeErrors:
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            trainWholeErr_n_30[n_30].append(trajErr[n_30])

In [ ]:
trainWholeErr_30NP = np.array(trainWholeErr_30)
trainWholeErr_60NP = np.array(trainWholeErr_60)
trainWholeErr_90NP = np.array(trainWholeErr_90)
trainWholeErr_120NP = np.array(trainWholeErr_120)
trainWholeErr_150NP = np.array(trainWholeErr_150)
trainWholeErr_180NP = np.array(trainWholeErr_180)
trainWholeErr_210NP = np.array(trainWholeErr_210)
trainWholeErr_240NP = np.array(trainWholeErr_240)

In [ ]:
trainWholeErrMean = [np.mean(trainWholeErr_30NP) \
            ,np.mean(trainWholeErr_60NP) \
            ,np.mean(trainWholeErr_90NP) \
            ,np.mean(trainWholeErr_120NP) \
            ,np.mean(trainWholeErr_150NP) \
            ,np.mean(trainWholeErr_180NP) \
            ,np.mean(trainWholeErr_210NP) \
            ,np.mean(trainWholeErr_240NP) \
           ]

In [ ]:
def format_func(value, tick_number):
    tempTick = (value*30) + 30
    ret = "%d"%(tempTick)
    return ret

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error Value On Training")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(trainWholeErrMean,label = "TS = %d"%(modelTS))
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
#compute errors for all testing trajectories
testDataWholeErrors = []
missedTrajCount = 0
# for vesselDataSource in vesselDataSourcesTest[0:1]:
for vesselDataSource in vesselDataSourcesTest[0:]:
    print("Taking data from:%s"%vesselDataSource.srcDir)
    mMSIListFile = vesselDataSource.srcDir + 'TrainTrajCount.txt'
    mMSIList = [line.rstrip('\n') for line in open(mMSIListFile)]
#     for traj in mMSIList[0:1]:
    for traj in mMSIList[0:]:
        numTraj = traj.split("_")[-1]
        trajAffix = traj[0:-len(numTraj)]
        for num in range(int(numTraj)):
            trajName = trajAffix + str(num) +'.csv'
            #TODO:.replace statement can be removed in the future
            testDataWholeErrors.append(get_error_for_traj(trajName.replace("jcharla","jagir"),vesselDataSource.type))

testWholeErr_30 = []
testWholeErr_60 = []
testWholeErr_90 = []
testWholeErr_120 = []
testWholeErr_150 = []
testWholeErr_180 = []
testWholeErr_210 = []
testWholeErr_240 = []

testWholeErr_n_30 = [testWholeErr_30 \
, testWholeErr_60 \
, testWholeErr_90 \
, testWholeErr_120 \
, testWholeErr_150 \
, testWholeErr_180 \
, testWholeErr_210 \
, testWholeErr_240 \
]

for trajErr in testDataWholeErrors:
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            testWholeErr_n_30[n_30].append(trajErr[n_30])
            
testWholeErr_30NP = np.array(testWholeErr_30)
testWholeErr_60NP = np.array(testWholeErr_60)
testWholeErr_90NP = np.array(testWholeErr_90)
testWholeErr_120NP = np.array(testWholeErr_120)
testWholeErr_150NP = np.array(testWholeErr_150)
testWholeErr_180NP = np.array(testWholeErr_180)
testWholeErr_210NP = np.array(testWholeErr_210)
testWholeErr_240NP = np.array(testWholeErr_240)

testWholeErrMean = [np.mean(testWholeErr_30NP) \
            ,np.mean(testWholeErr_60NP) \
            ,np.mean(testWholeErr_90NP) \
            ,np.mean(testWholeErr_120NP) \
            ,np.mean(testWholeErr_150NP) \
            ,np.mean(testWholeErr_180NP) \
            ,np.mean(testWholeErr_210NP) \
            ,np.mean(testWholeErr_240NP) \
           ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error Value On Testing")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(testWholeErrMean,label = "TS = %d"%(modelTS))
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
#save the error results
dataToStore = modelDir + "trainWholeErr_30NP.npy"
np.save(dataToStore, trainWholeErr_30NP)
dataToStore = modelDir + "trainWholeErr_60NP.npy"
np.save(dataToStore, trainWholeErr_60NP)
dataToStore = modelDir + "trainWholeErr_90NP.npy"
np.save(dataToStore, trainWholeErr_90NP)
dataToStore = modelDir + "trainWholeErr_120NP.npy"
np.save(dataToStore, trainWholeErr_120NP)
dataToStore = modelDir + "trainWholeErr_150NP.npy"
np.save(dataToStore, trainWholeErr_150NP)
dataToStore = modelDir + "trainWholeErr_180NP.npy"
np.save(dataToStore, trainWholeErr_180NP)
dataToStore = modelDir + "trainWholeErr_210NP.npy"
np.save(dataToStore, trainWholeErr_210NP)
dataToStore = modelDir + "trainWholeErr_240NP.npy"
np.save(dataToStore, trainWholeErr_240NP)

dataToStore = modelDir + "testWholeErr_30NP.npy"
np.save(dataToStore, testWholeErr_30NP)
dataToStore = modelDir + "testWholeErr_60NP.npy"
np.save(dataToStore, testWholeErr_60NP)
dataToStore = modelDir + "testWholeErr_90NP.npy"
np.save(dataToStore, testWholeErr_90NP)
dataToStore = modelDir + "testWholeErr_120NP.npy"
np.save(dataToStore, testWholeErr_120NP)
dataToStore = modelDir + "testWholeErr_150NP.npy"
np.save(dataToStore, testWholeErr_150NP)
dataToStore = modelDir + "testWholeErr_180NP.npy"
np.save(dataToStore, testWholeErr_180NP)
dataToStore = modelDir + "testWholeErr_210NP.npy"
np.save(dataToStore, testWholeErr_210NP)
dataToStore = modelDir + "testWholeErr_240NP.npy"
np.save(dataToStore, testWholeErr_240NP)